In [1]:
library(edgeR)
library(magrittr)
library(ggplot2)
library(DESeq2)

Loading required package: limma

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tap

# Automatise

# On cell-state level

## For LV

In [2]:
x=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_ALL_CELLTYPE_PSEUDOBULK_LV.csv")
genes_tofilter=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_ALL_CELLTYPE_PSEUDOBULK_FILTERING_LV.csv")

# Only needed for the column cell_state
#CELLTYPE_STATE <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_CELLSTATE_TRANSLATION_TABLE.csv")
#colnames(CELLTYPE_STATE) <- c("cell_states", 'cell_type')

# Removes patients with only few (<10) nuclei numbers (as they might bias the analysis)
CELLTYPE_FILTER <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_ABSOLUTE_CELLTYPE_NUMBER_LV.csv")


rownames(x) <- x$X
x <- x[,-1]
l <- strsplit(colnames(x), "__")

In [3]:
head(x)

,Control__AD__BS_H15,Control__AD__BS_H20,Control__AD__BS_H25,Control__AD__BS_H26,Control__AD__BS_H35,Control__AD__CB_D1_,Control__AD__CB_D2_,Control__AD__CB_D4_,Control__AD__CB_D5_,Control__AD__CB_D6_,⋯,Post.Vaccination__Mural__CVK1286,Post.Vaccination__Mural__CVK1336,Post.Vaccination__Myeloid__CVK1233,Post.Vaccination__Myeloid__CVK1281,Post.Vaccination__Myeloid__CVK1286,Post.Vaccination__Myeloid__CVK1336,Post.Vaccination__NC__CVK1233,Post.Vaccination__NC__CVK1281,Post.Vaccination__NC__CVK1286,Post.Vaccination__NC__CVK1336
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
FAM138A,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
OR4F5,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,1,0,4,14,10,6,31,2,0,⋯,2,3,8,2,7,10,1,0,2,1
AL627309.3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AL627309.2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [4]:
head(genes_tofilter)

,X,Control__AD__BS_H15,Control__AD__BS_H20,Control__AD__BS_H25,Control__AD__BS_H26,Control__AD__BS_H35,Control__AD__CB_D1_,Control__AD__CB_D2_,Control__AD__CB_D4_,Control__AD__CB_D5_,⋯,Post.Vaccination__Mural__CVK1286,Post.Vaccination__Mural__CVK1336,Post.Vaccination__Myeloid__CVK1233,Post.Vaccination__Myeloid__CVK1281,Post.Vaccination__Myeloid__CVK1286,Post.Vaccination__Myeloid__CVK1336,Post.Vaccination__NC__CVK1233,Post.Vaccination__NC__CVK1281,Post.Vaccination__NC__CVK1286,Post.Vaccination__NC__CVK1336
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,MIR1302-2HG,0,0.00000000,0,0.00000000,0.0000000,0.0,0.00000000,0.0000000,0.00000000,⋯,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0.00000000
2,FAM138A,0,0.00000000,0,0.00000000,0.0000000,0.0,0.00000000,0.0000000,0.00000000,⋯,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0.00000000
3,OR4F5,0,0.00000000,0,0.00000000,0.0000000,0.0,0.00000000,0.0000000,0.00000000,⋯,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0.00000000
4,AL627309.1,0,0.05555556,0,0.04081633,0.2372881,0.2,0.08955224,0.3131313,0.03333333,⋯,0.004366812,0.01694915,0.02228412,0.01020408,0.01639344,0.03460208,0.03448276,0,0.0338983,0.06666667
5,AL627309.3,0,0.00000000,0,0.00000000,0.0000000,0.0,0.00000000,0.0000000,0.00000000,⋯,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0.00000000
6,AL627309.2,0,0.00000000,0,0.00000000,0.0000000,0.0,0.00000000,0.0000000,0.00000000,⋯,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.0000000,0.00000000


In [6]:
meta.data <- t(as.data.frame(l))
rownames(meta.data) <- colnames(x)

meta.data <- as.data.frame(meta.data)
colnames(meta.data)  <-  c("Condition_custom", "cell_state", 'Patient')

In [8]:
GENOTYPES <- unique(meta.data[,"Condition_custom"][meta.data[,"Condition_custom"] != "Control"])
GENOTYPES

[1] Myocarditis      Pims             Post.Covid.19    Post.Vaccination
Levels: Control Myocarditis Pims Post.Covid.19 Post.Vaccination

In [9]:
CELL_LEVEL <- "CELLTYPES"
REGION <- "LV"

In [10]:
##########
# Convert to bool
#CELLTYPE_FILTER <- CELLTYPE_FILTER
CELLTYPE_FILTER[,2:ncol(CELLTYPE_FILTER)] <- apply(CELLTYPE_FILTER[,-1], 2, function(i) i>=5)

In [23]:
i <- 1

for(GENOTYPE in as.character(GENOTYPES)){
    message("\n###START: ", GENOTYPE," #####\n")
    
    for(CELL_STATE in as.character(unique(meta.data$cell_state))){
    #for(CELL_STATE in c("EC7")){
        
        CONTROL_COLUMN <- paste("control_",CELL_STATE, sep="")
        GENOTYPE_COLUMN <- paste(GENOTYPE,"_", CELL_STATE, sep="")
        
        #CELLTYPE <- as.character(CELLTYPE_STATE[which(CELLTYPE_STATE$cell_states==CELL_STATE),"cell_type"])
        
        
        x_sub <- x[,which(meta.data[,"Condition_custom"] %in% c("Control", GENOTYPE))]
        meta.data_sub <- meta.data[which(meta.data[,"Condition_custom"] %in% c("Control", GENOTYPE)),]
        
        x_sub <- x_sub[,which(meta.data_sub$cell_state %in% c(CELL_STATE))]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$cell_state %in% c(CELL_STATE)),]
        
        ##### remove first line (cell_type column)
        CELLTYPE_FILTER_SUB <- CELLTYPE_FILTER
        PATIENTS_TOKEEP <- CELLTYPE_FILTER_SUB[which(CELLTYPE_FILTER_SUB$celltype_classified==CELL_STATE),-1]
        PATIENTS_TOKEEP <- colnames(PATIENTS_TOKEEP)[which(as.logical(PATIENTS_TOKEEP))]
        #####
        
        
        
        
        message("Patient subsetting complete")
        
        
        
        # Subset only for patients which fulfill condition
        x_sub <- x_sub[,which(meta.data_sub$Patient %in% PATIENTS_TOKEEP)]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$Patient %in% PATIENTS_TOKEEP),]
        meta.data_sub[,"Condition_custom"] = droplevels(meta.data_sub[,"Condition_custom"]) ## Drop factor levels
   
        
        # The test is only run, if 2 conditions have at least 3 patients
        if (length(unique(meta.data_sub[,"Condition_custom"]))==2 & 
            all(table(as.character(meta.data_sub[,"Condition_custom"]))>=3))
           {
             # Prepare DGEList object
            meta.data_sub[,"Condition_custom"] <- as.factor(as.character(meta.data_sub[,"Condition_custom"]))
            dge <- DGEList(counts=x_sub, group=meta.data_sub[,"Condition_custom"])
            
            # Filter genes based on expression, the qlf Object will be filtered to exclude them from FDR calculation
            genes_tofilter_sub_GENOTYPE <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% GENOTYPE &
                                                                 gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                 gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]

            genes_tofilter_sub_CONTROL <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% 'Control' &
                                                                gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]
            
            genes_tofilter_sub <- data.frame("X"=genes_tofilter[,"X"],
                                             CONTROL_COLUMN=apply(genes_tofilter_sub_CONTROL, 1, mean),
                                             GENOTYPE_COLUMN=apply(genes_tofilter_sub_GENOTYPE, 1, mean))
            colnames(genes_tofilter_sub) <- c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)
            
            keep <- which(genes_tofilter_sub[,CONTROL_COLUMN] > 0.0125 |
                          genes_tofilter_sub[,GENOTYPE_COLUMN] > 0.0125)
            
            #keep <- genes_tofilter[,CONTROL_COLUMN]>0.0125 | ##### change & to | ##### 
            #        genes_tofilter[,GENOTYPE_COLUMN]>0.0125
            #dge <- dge[keep, , keep.lib.sizes=FALSE]
    
            # PP, model matrix, https://www.nature.com/articles/nmeth.4612 (edgeRQLFDetRate)
            #dge <- calcNormFactors(dge)
            #cdr <- scale(colMeans(x_sub > 0))
            #design <- model.matrix(~ cdr + meta.data_sub[,"Condition_custom"])
            design <- model.matrix(~ meta.data_sub[,"Condition_custom"])
    
            # 
            dge <- estimateDisp(dge, design = design)
            fit <- glmQLFit(dge, design = design)
            qlf <- glmQLFTest(fit)
    
            # For all
            tt <- topTags(qlf, n = Inf)
            
            # Only for "expressed/detected" genes
            tt_filtered <- topTags(qlf[keep,], n = Inf)
            
            tt$table[,"Gene"] <- as.character(rownames(tt$table))
            tt_filtered$table[,"Gene"] <- as.character(rownames(tt_filtered$table))
            tt_filtered$table[,"low_expression"] <- "F" #####  Low-expression column ##### 
            tt_filtered$table[,"FDR_plot"] <- -log10(tt_filtered$table$FDR) #####  For plotting Volcano, here we use FDR, not pValue  ##### 
            
            
            tmp <- tt$table[,c("Gene", "logFC", "logCPM", 'F', 'PValue', 'FDR')]
            colnames(tmp) <- gsub("FDR", "FDR_nofilter", colnames(tmp))
            
            
            tt_merged <- merge(tmp, 
                               tt_filtered$table[,c("Gene", "FDR", "low_expression", "FDR_plot")], 
                               by="Gene", all=T)   
            
            # Replace NA FDRs with NA                   #####  ORDER CHANGED #####  
            tt_merged[which(is.na(tt_merged$FDR)),"FDR"] <- 1 
            
            EXPRESSION_MEAN <- genes_tofilter_sub[,c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)]
            colnames(EXPRESSION_MEAN) <- c("Gene", "mean_exp_control", 'mean_exp_genotype')
            tt_merged <- merge(tt_merged, EXPRESSION_MEAN, by="Gene")
    
            
            tt_merged[,"Region"] <- REGION
            tt_merged[,"annotation_level"] <- CELL_LEVEL
            tt_merged[,"cell_state"] <- NA
            tt_merged[,"cell_type"] <- CELL_STATE
            
            
            tt_merged[,"comparison"] <- paste("control_", GENOTYPE, sep="")
            
            ###### Suggestion to add those columns
            tt_merged[,"Observations_genotype"] <- as.numeric(table(meta.data_sub[,"Condition_custom"])[GENOTYPE])
            tt_merged[,"Observations_reference"] <- as.numeric(table(meta.data_sub[,"Condition_custom"])['Control'])
            
            # low expression add T
            tt_merged[which(is.na(tt_merged$low_expression)),"low_expression"] <- "T"
            
            if(i==1){
                 final_df <- tt_merged   
            } else {
                final_df <- rbind(final_df, tt_merged)
            }
            
            i <- i + 1
            message("\n###FINISHED: ", CELL_STATE," #####\n")
        } else {
            message("\n###Skip: ", CELL_STATE," #####\n")
        }
    }
}


###START: Myocarditis #####


Patient subsetting complete


###Skip: AD #####


Patient subsetting complete


###FINISHED: CM #####


Patient subsetting complete


###FINISHED: EC_PKHD1L1. #####


Patient subsetting complete


###FINISHED: EC #####


Patient subsetting complete


###FINISHED: FB #####


Patient subsetting complete


###FINISHED: Lymphoid #####


Patient subsetting complete


###Skip: Mast #####


Patient subsetting complete


###FINISHED: Mural #####


Patient subsetting complete


###FINISHED: Myeloid #####


Patient subsetting complete


###FINISHED: NC #####



###START: Pims #####


Patient subsetting complete


###Skip: AD #####


Patient subsetting complete


###Skip: CM #####


Patient subsetting complete


###Skip: EC_PKHD1L1. #####


Patient subsetting complete


###Skip: EC #####


Patient subsetting complete


###Skip: FB #####


Patient subsetting complete


###Skip: Lymphoid #####


Patient subsetting complete


###Skip: Mast #####


Patient subsetting co

In [27]:
write.csv(final_df, "/fast/AG_Huebner/huebner3/ANALYSES/20200706_el_COVID_Biopsy/data/COVID_ALLCELLTYPES_ALLGENOTYPES_EDGER_LV_V4_NOTTM.csv")